In [1]:
import qiskit
from qiskit import assemble, QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile
from random import *
from qiskit import Aer
sim = Aer.get_backend("statevector_simulator")
import numpy as np
import pandas as pd
from qiskit.circuit.library.standard_gates import HGate
from qiskit.circuit.library import RZGate, RYGate
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
def neighbor_list(i, j, N):
  a = i-1
  b = j-1
  c = i+1 
  d = j+1 
  if a < 0: 
      a = N-1
  if b < 0: 
      b = N-1
  if c > N-1: 
      c = 0 
  if d > N-1: 
      d = 0
  left   = (a, j)
  above  = (i, b)
  right  = (c, j)
  below  = (i, d)
  return [left, above, right, below]

In [3]:
spin, N = 16, 4

In [4]:
def single_step_qc(i, temp): 
    global N, spin 
    prob1= np.exp(-8/temp)
    angle1 = 2*np.arccos(np.sqrt(prob1))
    prob2 = np.exp(-4/temp)
    angle2 = 2*np.arccos(np.sqrt(prob2))
    ### construct neighbor qubit 
    ising = np.arange(0, N**2).reshape(N,N)
    index = np.where(i==ising)
    neighbor_qubit = []
    for k in neighbor_list(index[0], index[1],N):
        neighbor_qubit.append(ising[k][0])
    ### construct circuit 
    qc = QuantumCircuit(spin+4)
    aux_qubit = [spin, spin+1, spin+2, spin+3]
    # flip first 
    qc.x(i)
    qc.barrier()
    for j in range(4): 
        qc.cx(i, aux_qubit[j])
        qc.cx(neighbor_qubit[j], aux_qubit[j])
    # rotate or not depends on sign
    #####################situation 1 : all same signs #####################
    [aux1, aux2, aux3, aux4] = aux_qubit
    # default is ferromagnetic
    mcry= RYGate(angle1).control(4,label=None )
    qc.append(mcry,[aux1, aux2, aux3, aux4, i])
    #####################situation 2 : 3 same signs, 1 different signs #####################
    mcry_1= RYGate(angle2).control(4,label=None )
    mcry_2= RYGate(angle2).control(4,label=None )
    mcry_3= RYGate(angle2).control(4,label=None )
    mcry_4= RYGate(angle2).control(4,label=None )
    mcry_1._ctrl_state=14
    mcry_2._ctrl_state=13
    mcry_3._ctrl_state=11
    mcry_4._ctrl_state=7
    qc.append(mcry_1, [aux1, aux2, aux3, aux4, i])
    qc.append(mcry_2, [aux1, aux2, aux3, aux4, i])
    qc.append(mcry_3, [aux1, aux2, aux3, aux4, i])
    qc.append(mcry_4, [aux1, aux2, aux3, aux4, i])
    qc.barrier()
    return qc

In [5]:
def cal_M(state):
    M = 0
    for i in state:
        if i == '1': 
            M += 1
        else: 
            M -= 1 
    return M/16

In [6]:
def phase_space_sweep(temp, circuit_size = 32):
    # MC_sweep
    qc = QuantumCircuit(spin+4)
    # for 4x4 ising model at least need to repeat 16 circuits to get all basis (2**16)
    # I used 32 to make sure that we can cover the whole phase space
    # could change this number
    for _ in range(circuit_size):
        pick = randint(0, spin-1)
        qc.compose(single_step_qc(pick, temp), inplace=True)
    return qc

In [7]:
def measure(circuit):
    circuit.remove_final_measurements()
    circuit.measure_all()
    backend = Aer.get_backend('aer_simulator')
    circuit = transpile(circuit, backend )
    final_counts = backend.run(circuit, shots=8192).result().get_counts()
    print(final_counts)
    state = max(final_counts, key= final_counts.get)[4:]
    mag = abs(cal_M(state))
    print("Most probably state: ", state)
    print("Magnetization: ", mag) 
    return state, mag

In [8]:
for _ in range(5):
    qc = phase_space_sweep(4)
    measure(qc)

Most probably state:  0111111100010000
Magnetization:  0.0
Most probably state:  0001111110110010
Magnetization:  0.125
Most probably state:  1100110001100101
Magnetization:  0.0
Most probably state:  0100101110101110
Magnetization:  0.125
Most probably state:  0001110110111111
Magnetization:  0.375


In [12]:
# Average of many results from circuit size 32
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

for temp in temps:
    magnetization = []
    for i in range(10):
        qc = phase_space_sweep(temp)
        state, mag = measure(qc)
        magnetization.append(mag)
    df['kT='+str(temp)] = magnetization

df.to_csv('Circuit 32_Samples 10_statevector.csv')

Most probably state:  0111111111100010
Magnetization:  0.375
Most probably state:  0100010010101111
Magnetization:  0.0
Most probably state:  0110110001001100
Magnetization:  0.125
Most probably state:  0110000001110111
Magnetization:  0.0
Most probably state:  0000000000110101
Magnetization:  0.5
Most probably state:  1011110000110110
Magnetization:  0.125
Most probably state:  1011001101001011
Magnetization:  0.125
Most probably state:  0100001000111100
Magnetization:  0.25
Most probably state:  0100010110100001
Magnetization:  0.25
Most probably state:  1010100110000000
Magnetization:  0.375
Most probably state:  1011000101001111
Magnetization:  0.125
Most probably state:  0001101010100011
Magnetization:  0.125
Most probably state:  1100000011100001
Magnetization:  0.25
Most probably state:  0101001010100100
Magnetization:  0.25
Most probably state:  0011010001110111
Magnetization:  0.125
Most probably state:  1001101101101000
Magnetization:  0.0
Most probably state:  00010101000100

Most probably state:  1101011011011000
Magnetization:  0.125
Most probably state:  0000001101100010
Magnetization:  0.375
Most probably state:  0010100011010101
Magnetization:  0.125
Most probably state:  1100101101010100
Magnetization:  0.0
Most probably state:  1110011011010100
Magnetization:  0.125
Most probably state:  1001100001110011
Magnetization:  0.0
Most probably state:  1001111101111101
Magnetization:  0.5
Most probably state:  1001100001010010
Magnetization:  0.25
Most probably state:  0011001011001010
Magnetization:  0.125
Most probably state:  0000111110101001
Magnetization:  0.0
Most probably state:  0111011011011111
Magnetization:  0.5
Most probably state:  1101000111101100
Magnetization:  0.125
Most probably state:  0110001111110100
Magnetization:  0.125
Most probably state:  1000001100101011
Magnetization:  0.125
Most probably state:  1011011100110100
Magnetization:  0.125
Most probably state:  1011001111000101
Magnetization:  0.125
Most probably state:  1110101001001

In [9]:
# Average of many results from circuit size 32
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

for temp in temps:
    magnetization = []
    for i in range(10):
        qc = phase_space_sweep(temp, 64)
        state, mag = measure(qc)
        magnetization.append(mag)
    df['kT='+str(temp)] = magnetization

df.to_csv('Circuit 64_Samples 10_statevector.csv')

Most probably state:  0000111111000101
Magnetization:  0.0
Most probably state:  1100000001011001
Magnetization:  0.25
Most probably state:  1100010010001001
Magnetization:  0.25
Most probably state:  1000101001001011
Magnetization:  0.125
Most probably state:  1101011100101111
Magnetization:  0.375
Most probably state:  1001000001111110
Magnetization:  0.0
Most probably state:  0101000010110010
Magnetization:  0.25
Most probably state:  0011000011010100
Magnetization:  0.25
Most probably state:  1001000001000000
Magnetization:  0.625
Most probably state:  1001101101010000
Magnetization:  0.125
Most probably state:  0100101010100010
Magnetization:  0.25
Most probably state:  1010010110011011
Magnetization:  0.125
Most probably state:  0111100101010011
Magnetization:  0.125
Most probably state:  0100000101100110
Magnetization:  0.25
Most probably state:  1110101011001100
Magnetization:  0.125
Most probably state:  0101010110010011
Magnetization:  0.0
Most probably state:  00100101001101

Most probably state:  1110010000100101
Magnetization:  0.125
Most probably state:  0000100001010100
Magnetization:  0.5
Most probably state:  0110001101001111
Magnetization:  0.125
Most probably state:  0000100100100110
Magnetization:  0.375
Most probably state:  1110011000011010
Magnetization:  0.0
Most probably state:  1101010111100011
Magnetization:  0.25
Most probably state:  1101011101101110
Magnetization:  0.375
Most probably state:  1000111111001111
Magnetization:  0.375
Most probably state:  0110001101011111
Magnetization:  0.25
Most probably state:  1111011000110110
Magnetization:  0.25
Most probably state:  1011111110010010
Magnetization:  0.25
Most probably state:  1000101001111100
Magnetization:  0.0
Most probably state:  1000001100111111
Magnetization:  0.125
Most probably state:  1111000011001001
Magnetization:  0.0
Most probably state:  1100011010110001
Magnetization:  0.0
Most probably state:  1011101101111001
Magnetization:  0.375
Most probably state:  1010100101011001

In [10]:
#Circuit size 256
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

magnetization = []
for temp in temps:
    qc = phase_space_sweep(temp, 256)
    state, mag = measure(qc)
    magnetization.append(mag)
df['kT'] = temps
df['Magnetization'] = magnetization

df.to_csv('Circuit 256_Samples 1_statevector.csv')

Most probably state:  1010100100101011
Magnetization:  0.0
Most probably state:  1010111100011001
Magnetization:  0.125
Most probably state:  1100100100001001
Magnetization:  0.25
Most probably state:  1111101111001001
Magnetization:  0.375
Most probably state:  1100001000001100
Magnetization:  0.375
Most probably state:  0000111100000110
Magnetization:  0.25
Most probably state:  1111001110000110
Magnetization:  0.125
Most probably state:  0010101001101011
Magnetization:  0.0
Most probably state:  0101000100000010
Magnetization:  0.5
Most probably state:  0100101010000010
Magnetization:  0.375
Most probably state:  0100111011010111
Magnetization:  0.25
Most probably state:  0100111010001110
Magnetization:  0.0
Most probably state:  0101011110010111
Magnetization:  0.25
Most probably state:  1000000001101110
Magnetization:  0.25
Most probably state:  1111001011111100
Magnetization:  0.375
Most probably state:  0100111000111000
Magnetization:  0.125
Most probably state:  110001010001010

In [11]:
#Circuit size 512
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

magnetization = []
for temp in temps:
    qc = phase_space_sweep(temp, 512)
    state, mag = measure(qc)
    magnetization.append(mag)
df['kT'] = temps
df['Magnetization'] = magnetization

df.to_csv('Circuit 512_Samples 1_statevector.csv')

Most probably state:  1111100111000001
Magnetization:  0.125
Most probably state:  0001011010011001
Magnetization:  0.125
Most probably state:  1101011111001110
Magnetization:  0.375
Most probably state:  1111010101001110
Magnetization:  0.25
Most probably state:  0111011110000011
Magnetization:  0.125
Most probably state:  1101001111000100
Magnetization:  0.0
Most probably state:  1111010100101011
Magnetization:  0.25
Most probably state:  1101010000101000
Magnetization:  0.25
Most probably state:  0001101010011100
Magnetization:  0.125
Most probably state:  0011110001000001
Magnetization:  0.25
Most probably state:  0101000101100011
Magnetization:  0.125
Most probably state:  1110110111010010
Magnetization:  0.25
Most probably state:  1000110111100010
Magnetization:  0.0
Most probably state:  0110000111010100
Magnetization:  0.125
Most probably state:  0000100111011000
Magnetization:  0.25
Most probably state:  0111000111001011
Magnetization:  0.125
Most probably state:  101001110110

In [8]:
%%time
qc = phase_space_sweep(4, 512)
qc = transpile(qc,sim)
final_counts = sim.run(qc, shots=1).result().get_counts()
prob, state = max(final_counts.values()), max(final_counts, key = final_counts.get)
mag = abs(cal_M(state))
plot_histogram(final_counts)
print(mag)

0.125
CPU times: user 25min 31s, sys: 7min 20s, total: 32min 52s
Wall time: 47min 54s


In [ ]:
plot_histogram(final_counts)